In [1]:
!rm -r /content/NLP-DL-Group2/
!cd /content && git clone -b fnn https://github.com/AnshulH/NLP-DL-Group2

rm: cannot remove '/content/NLP-DL-Group2/': No such file or directory
Cloning into 'NLP-DL-Group2'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 90 (delta 17), reused 20 (delta 5), pack-reused 46
Unpacking objects: 100% (90/90), 30.53 MiB | 2.52 MiB/s, done.
Updating files: 100% (20/20), done.


In [2]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import math
import time
import numpy as np
import sys
import argparse
import os
import re
from nltk.util import ngrams

In [26]:
def decode(vocab,corpus):
    
    text = ''
    for i in range(len(corpus)):
        wID = corpus[i]
        text = text + vocab[wID] + ' '
    return(text)

def encode(words,text):
    corpus = []
    tokens = text.split(' ')
    for t in tokens:
        try:
            wID = words[t][0]
        except:
            wID = words['<unk>'][0]
        corpus.append(wID)
    return(corpus)

def read_encode(file_name,vocab,words,corpus,threshold):
    
    wID = len(vocab)
    
    if threshold > -1:
        with open(file_name,'rt', encoding='utf8') as f:
            for line in f:
                line = line.replace('\n','')
                # Added lower-casing
                line = line.lower()
                
                # Strips out all charcters other than alphanumeric
                line = re.sub('[\W_]+', ' ', line, flags=re.UNICODE)
                
                # Strips out numbers
                line = re.sub('\d+', '', line)
                
                tokens = line.split(' ')
                for t in tokens:
                    try:
                        elem = words[t]
                    except:
                        elem = [wID,0]
                        vocab.append(t)
                        wID = wID + 1
                    elem[1] = elem[1] + 1
                    words[t] = elem

        temp = words
        words = {}
        vocab = []
        wID = 0
        words['<unk>'] = [wID,100]
        vocab.append('<unk>')
        for t in temp:
            if temp[t][1] >= threshold:
                vocab.append(t)
                wID = wID + 1
                words[t] = [wID,temp[t][1]]
            
                    
    with open(file_name,'rt', encoding='utf8') as f:
        for line in f:
            line = line.replace('\n','')
            tokens = line.split(' ')
            for t in tokens:
                try:
                    wID = words[t][0]
                except:
                    wID = words['<unk>'][0]
                corpus.append(wID)
                
    return [vocab,words,corpus]

In [31]:
lookup = dict()
print(words)
for key in words:
  lookup[words[key][0]] = key
len(lookup)

{'<unk>': [0, 100], '': [1, 523021], 'start': [2, 8044], 'bio': [3, 15947], 'hildebrand': [4, 23], 'bothe': [5, 15], 'october': [6, 1575], 'september': [7, 1388], 'was': [8, 55929], 'a': [9, 57759], 'swiss': [10, 327], 'botanist': [11, 1290], 'born': [12, 7191], 'in': [13, 107962], 'strasbourg': [14, 264], 'and': [15, 83715], 'studied': [16, 3990], 'medicine': [17, 1486], 'stuttgart': [18, 220], 'bonn': [19, 204], 'with': [20, 11153], 'dissertation': [21, 397], 'on': [22, 19010], 'plants': [23, 873], 'collected': [24, 388], 'at': [25, 29192], 'the': [26, 163887], 'trier': [27, 30], 'sent': [28, 316], 'by': [29, 8263], 'natural': [30, 2186], 'history': [31, 3383], 'collection': [32, 847], 'of': [33, 135798], 'london': [34, 3938], 'society': [35, 6849], 'to': [36, 38688], 'belgium': [37, 114], 'he': [38, 61656], 'became': [39, 8032], 'plant': [40, 973], 'pathologist': [41, 113], 'started': [42, 628], 'work': [43, 5678], 'as': [44, 17147], 'taxonomist': [45, 41], 'botanical': [46, 1113], 

29406

In [4]:
params = {
        'd_model': 100,
        'd_hidden': 100,
        'n_layers': 2,
        'batch_size': 20,
        'seq_len': 30,
        'printevery': 5000,
        'window': 3,
        'epochs': 20,
        'lr': 0.0001,
        'dropout': 0.35,
        'clip': 2.0,
        'model': 'FFNN',
        'savename': 'lstm',
        'loadname': None,
        'trainname': '/content/NLP-DL-Group2/hw#1/fake.train.txt',
        'validname': '/content/NLP-DL-Group2/hw#1/fake.valid.txt',
        'testname': '/content/NLP-DL-Group2/hw#1/fake.test.txt'
    }
torch.manual_seed(0)

[vocab,words,traim] = read_encode(params['trainname'],[],{},[],3)
print('vocab: %d train: %d' % (len(vocab), len(train)))
[vocab,words,test] = read_encode(params['testname'], vocab,words,[],-1)
print('vocab: %d test: %d' % (len(vocab),len(test)))
params['vocab_size'] = len(vocab)

vocab: 29406 train: 3275943
vocab: 29406 test: 424931


In [5]:
# Returns bios in [(bio without puncutation, label), ...]
# 0: FAKE
# 1: REAL
def read_bios(file_name, labels=False):
    with open(file_name,'rt', encoding='utf8') as f:
        all_bios = f.readlines()
        
    split_bios = []
    curr_bio = ""
    curr_index = 0
    while curr_index < len(all_bios):
        curr_line = all_bios[curr_index].lower()
        # Strips out all charcters other than alphanumeric
        curr_line = re.sub('[\W_]+', ' ', curr_line, flags=re.UNICODE)
        
        # Strips out numbers
        curr_line = re.sub('\d+', '', curr_line)
        
        curr_line = curr_line.strip()
        
        if curr_line == "start bio":
            # Skips their name
            curr_index += 1
        
        elif curr_line == "end bio":
          if labels:
            curr_index += 2
            if "FAKE" in all_bios[curr_index]:
                label = 0
            else:
                label = 1

                split_bios.append((curr_bio, label))
                curr_bio = ""
            
          else:
            split_bios.append((curr_bio, 0))
            curr_bio = ""
        
        else:
            # Check to ensure not empty space
            if curr_line:
                if curr_bio == "":
                    curr_bio = curr_line
                else:
                    curr_bio += " " + curr_line
        
        curr_index += 1
        
    
    return split_bios
    
split_bios = read_bios(params['trainname'], False)

In [6]:
# Create windows
# split_bios: [(bio without puncutation, label), ...]
# Returns sliding windows (multiple per biography):
# [
#  [
#   (['Hildebrand', 'Bothe', 'October'], 'September'),
#   (['Bothe', 'October', 'September'], 'was'),
#  ],
#  [
#   ([Hermann', 'Robert', 'Kaiser'], 'September'),
#   ...
#  ]
# ]

# Reference: https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html
def create_windows(split_bios, window_size):
    sliding_windows = []
    for bio, label in split_bios:
        bio_without_nums = ''.join([i for i in bio if not i.isdigit()])
        tokens = [token for token in bio_without_nums.split(" ") if token != ""]
        
        ngrams = []
        for i in range(len(tokens) - window_size):
            ngrams.append((
                [tokens[i + j] for j in range(window_size)],
                tokens[i + window_size]
            ))
        
        sliding_windows.append((ngrams, label))
    
    return sliding_windows

windows = create_windows(split_bios, 3)

In [7]:
def create_context_and_next_words(windows):
  all_context = []
  all_next_words = []
  skipped_labels = 0

  for (each_bio, bio_label) in windows:
      for context, label in each_bio:
          found_in_words = [word in words for word in context]
          found_in_words.extend([label in words])
          if all(found_in_words):
              all_context.append([words[word][0] for word in context])
              all_next_words.append([words[label][0]])
          else:
              all_context.append([0] * len(context))
              all_next_words.append([0])
              
              skipped_labels += 1

  return torch.LongTensor(all_context), torch.LongTensor(all_next_words)

# Skipping certain sliding windows because they weren't found in the dictionary
# Unk'd?
# print(skipped_labels)

all_context, all_next_words = create_context_and_next_words(windows)

In [8]:
import torch
from torch.utils.data import DataLoader, TensorDataset

BATCH_SIZE = 64

dataset = TensorDataset(all_context, all_next_words)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [23]:
class FFNN(nn.Module):
    # d_model = embedding dimensions
    def __init__(self, vocab, words,d_model, d_hidden, dropout):
        super().__init__() 
    
        self.vocab = vocab
        self.words = words
        self.vocab_size = len(self.vocab)
        self.d_model = d_model
        self.d_hidden = d_hidden
        self.dropout = dropout
        self.embeds = nn.Embedding(self.vocab_size,d_model)
        
        # Context size * dimensions for input
        # Hidden layer neurons was more difficult to find information
        self.linear1 = nn.Linear(3 * d_model, 512)
        self.linear2 = nn.Linear(512, self.vocab_size)

    def forward(self, src):
        embeds = self.embeds(src).view((BATCH_SIZE, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs
    
    def lookup(src):
        embeds = self.embeds(src).view((BATCH_SIZE, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        return out
                
    def init_weights(self):
        pass

In [24]:
# AFTER BATCHING

model = FFNN(vocab, words, d_model=100, d_hidden=100, dropout=0.1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

checkpoint = torch.load('/content/gdrive/My Drive/fnn.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
model.eval()

# model = FFNN(vocab, words, d_model=100, d_hidden=100, dropout=0.1)
loss_function = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1):
    running_loss = 0
    for i, (context, label) in enumerate(dataloader):
        log_probabilities = model(context)
        # Collapsing labels to correct dimensions
        label = label.squeeze()
        loss = loss_function(log_probabilities, label)
        
        loss.backward()
        optimizer.step()
        
        model.zero_grad()
        
        running_loss += loss.item()

        if i % 200 == 199:
            last_loss = running_loss / 200 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            running_loss = 0.
            
            torch.save({
                'epoch': epoch,
                'batch': i,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
                }, "/content/gdrive/My Drive/fnn.pt")
            
            break
        
    print("Epoch {}/{}: Loss {:.4f}".format(epoch, 10, loss))
    
print("Finished!")

  batch 200 loss: 6.535521628856659
Epoch 0/10: Loss 5.9120
Finished!


In [46]:
import torch
from torch.utils.data import DataLoader, TensorDataset

split_bios = read_bios('/content/NLP-DL-Group2/hw#1/mix.train.txt', True)
windows = create_windows(split_bios, 3)

all_sliding_windows = [item[0] for item in windows]
all_labels = [item[1] for item in windows]

BATCH_SIZE = 64

# dataset = TensorDataset(all_sliding_windows, all_labels)
# dataloader = DataLoader(dataset, batch_size=BATCH_SIZE)

ValueError: ignored

In [ ]:
probabilities = []
for i, (context, label) in enumerate(dataloader):
  probability_table = model.lookup(context)
  probabilities.append(probability_table[label])